In [ ]:
import pandas as pd
import pickle

korea_df = pd.DataFrame()
china_df = pd.DataFrame()
japan_df = pd.DataFrame()

nations = ['korea', 'china', 'japan']
emotion_labels = ['joy', 'sadness', 'anger', 'fear', 'disgust', 'surprise', 'neutral']

for nation in nations:
    with open("reddit_" + nation + ".pickle", "rb") as fi:
        if nation == 'korea':
            korea_df = pickle.load(fi)
        elif nation == 'china':
            china_df = pickle.load(fi)
        elif nation == 'japan':
            japan_df = pickle.load(fi)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# 시작 날짜와 끝 날짜를 지정합니다.
start_date = '2018-01-01'
end_date = '2023-12-31'
valid_labels = ['joy', 'sadness', 'anger', 'fear', 'disgust', 'surprise', 'neutral']
scaler = MinMaxScaler()

# 각 국가별 데이터 프레임 초기화
korea_monthly_scaled = pd.DataFrame()
china_monthly_scaled = pd.DataFrame()
japan_monthly_scaled = pd.DataFrame()

# 각 국가별 데이터 프레임 처리
for nation, df in [('korea', korea_df), ('china', china_df), ('japan', japan_df)]:
    df['created_utc'] = pd.to_datetime(df['created_utc'], errors='coerce')
    df = df[(df['created_utc'] >= start_date) & (df['created_utc'] <= end_date)]
    df['year_month'] = df['created_utc'].dt.to_period('M')
    df['weighted_score'] = df['score']
    df = df[df['label'].isin(valid_labels)]
    
    if df.empty:
        continue
    
    monthly_weighted = df.groupby(['year_month', 'label'])['weighted_score'].sum().unstack(fill_value=0)
    monthly_scaled = pd.DataFrame(scaler.fit_transform(monthly_weighted),
                                  index=monthly_weighted.index,
                                  columns=monthly_weighted.columns)
    
    monthly_scaled.sort_index(inplace=True)  # 월별로 정렬
    
    if nation == 'korea':
        korea_monthly_scaled = monthly_scaled
    elif nation == 'china':
        china_monthly_scaled = monthly_scaled
    elif nation == 'japan':
        japan_monthly_scaled = monthly_scaled

# 감정별 월별 가중치 점수 합산
emotion_monthly_scaled = korea_monthly_scaled.add(china_monthly_scaled, fill_value=0).add(japan_monthly_scaled, fill_value=0)

# 그래프 설정
plt.figure(figsize=(14, 8))
emotion_monthly_scaled.plot(kind='line', marker='o', ax=plt.gca())

# 배경 색상 설정
ax = plt.gca()
ax.axvspan('2018-01', '2020-01', color='lightblue', alpha=0.3)
ax.axvspan('2020-01', '2022-01', color='lightpink', alpha=0.3)
ax.axvspan('2022-01', '2023-12', color='lightgreen', alpha=0.3)


# 그래프 제목 및 축 레이블 설정
plt.title('Monthly Weighted Scores of Emotions (Scaled)')
plt.xlabel('Year-Month')
plt.ylabel('Scaled Weighted Score')
plt.legend(title='Emotion')
plt.grid(True)

# x축 레이블을 45도 회전
plt.xticks(rotation=45)

# 그래프 출력
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# 시작 날짜와 끝 날짜를 지정합니다.
start_date = '2018-01-01'
end_date = '2023-12-31'
valid_labels = ['surprise']  # 표시할 감정을 'surprise'로 변경

# 월별 인덱스 생성
monthly_index = pd.period_range(start=start_date, end=end_date, freq='M')

# 각 국가별 데이터 프레임 초기화
korea_monthly_scaled = pd.DataFrame(index=monthly_index)
china_monthly_scaled = pd.DataFrame(index=monthly_index)
japan_monthly_scaled = pd.DataFrame(index=monthly_index)

# 각 국가별 데이터 프레임 처리
for nation, df in [('korea', korea_df), ('china', china_df), ('japan', japan_df)]:
    df['created_utc'] = pd.to_datetime(df['created_utc'], errors='coerce')
    df = df[(df['created_utc'] >= start_date) & (df['created_utc'] <= end_date)]
    df['year_month'] = df['created_utc'].dt.to_period('M')
    df['weighted_score'] = df['score']
    df = df[df['label'].isin(valid_labels)]
    
    if df.empty:
        print(f"No data for {nation} with label {valid_labels}")
        continue
    
    monthly_weighted = df.groupby(['year_month', 'label'])['weighted_score'].sum().unstack(fill_value=0)
    monthly_scaled = pd.DataFrame(MinMaxScaler().fit_transform(monthly_weighted),
                                  index=monthly_weighted.index,
                                  columns=monthly_weighted.columns)
    
    # 월별 인덱스와 병합
    monthly_scaled = monthly_scaled.reindex(monthly_index, fill_value=0)
    
    if nation == 'korea':
        korea_monthly_scaled = monthly_scaled
    elif nation == 'china':
        china_monthly_scaled = monthly_scaled
    elif nation == 'japan':
        japan_monthly_scaled = monthly_scaled

# 그래프 설정
plt.figure(figsize=(14, 8))

# 국가별 anger 감정의 월별 가중치 점수 그래프 그리기
if not korea_monthly_scaled.empty:
    plt.plot(korea_monthly_scaled.index.astype(str), korea_monthly_scaled['surprise'], marker='o', label='Korea')
if not china_monthly_scaled.empty:
    plt.plot(china_monthly_scaled.index.astype(str), china_monthly_scaled['surprise'], marker='o', label='China')
if not japan_monthly_scaled.empty:
    plt.plot(japan_monthly_scaled.index.astype(str), japan_monthly_scaled['surprise'], marker='o', label='Japan')

# 배경 색상 설정
ax = plt.gca()
ax.axvspan('2018-01', '2020-01', color='lightblue', alpha=0.3)
ax.axvspan('2020-01', '2022-01', color='lightpink', alpha=0.3)
ax.axvspan('2022-01', '2023-12', color='lightgreen', alpha=0.3)

# 그래프 제목 및 축 레이블 설정
plt.title('Monthly Weighted Scores of Surprise (Scaled)')
plt.xlabel('Year-Month')
plt.ylabel('Scaled Weighted Score')
plt.legend(title='Country')
plt.grid(True)

# x축 레이블을 90도 회전
plt.xticks(rotation=90)

# 그래프 출력
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# 시작 날짜와 끝 날짜를 지정합니다.
start_date = '2018-01-01'
end_date = '2023-12-31'
valid_labels = ['sadness']  # 표시할 감정을 'surprise'로 변경

# 월별 인덱스 생성
monthly_index = pd.period_range(start=start_date, end=end_date, freq='M')

# 각 국가별 데이터 프레임 초기화
korea_monthly_scaled = pd.DataFrame(index=monthly_index)
china_monthly_scaled = pd.DataFrame(index=monthly_index)
japan_monthly_scaled = pd.DataFrame(index=monthly_index)

# 각 국가별 데이터 프레임 처리
for nation, df in [('korea', korea_df), ('china', china_df), ('japan', japan_df)]:
    df['created_utc'] = pd.to_datetime(df['created_utc'], errors='coerce')
    df = df[(df['created_utc'] >= start_date) & (df['created_utc'] <= end_date)]
    df['year_month'] = df['created_utc'].dt.to_period('M')
    df['weighted_score'] = df['score']
    df = df[df['label'].isin(valid_labels)]
    
    if df.empty:
        print(f"No data for {nation} with label {valid_labels}")
        continue
    
    monthly_weighted = df.groupby(['year_month', 'label'])['weighted_score'].sum().unstack(fill_value=0)
    monthly_scaled = pd.DataFrame(MinMaxScaler().fit_transform(monthly_weighted),
                                  index=monthly_weighted.index,
                                  columns=monthly_weighted.columns)
    
    # 월별 인덱스와 병합
    monthly_scaled = monthly_scaled.reindex(monthly_index, fill_value=0)
    
    if nation == 'korea':
        korea_monthly_scaled = monthly_scaled
    elif nation == 'china':
        china_monthly_scaled = monthly_scaled
    elif nation == 'japan':
        japan_monthly_scaled = monthly_scaled

# 그래프 설정
plt.figure(figsize=(14, 8))

# 국가별 anger 감정의 월별 가중치 점수 그래프 그리기
if not korea_monthly_scaled.empty:
    plt.plot(korea_monthly_scaled.index.astype(str), korea_monthly_scaled['sadness'], marker='o', label='Korea')
if not china_monthly_scaled.empty:
    plt.plot(china_monthly_scaled.index.astype(str), china_monthly_scaled['sadness'], marker='o', label='China')
if not japan_monthly_scaled.empty:
    plt.plot(japan_monthly_scaled.index.astype(str), japan_monthly_scaled['sadness'], marker='o', label='Japan')

# 배경 색상 설정
ax = plt.gca()
ax.axvspan('2018-01', '2020-01', color='lightblue', alpha=0.3)
ax.axvspan('2020-01', '2022-01', color='lightpink', alpha=0.3)
ax.axvspan('2022-01', '2023-12', color='lightgreen', alpha=0.3)

# 그래프 제목 및 축 레이블 설정
plt.title('Monthly Weighted Scores of Sadness (Scaled)')
plt.xlabel('Year-Month')
plt.ylabel('Scaled Weighted Score')
plt.legend(title='Country')
plt.grid(True)

# x축 레이블을 90도 회전
plt.xticks(rotation=90)

# 그래프 출력
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# 시작 날짜와 끝 날짜를 지정합니다.
start_date = '2018-01-01'
end_date = '2023-12-31'
valid_labels = ['joy']  # 표시할 감정을 'surprise'로 변경

# 월별 인덱스 생성
monthly_index = pd.period_range(start=start_date, end=end_date, freq='M')

# 각 국가별 데이터 프레임 초기화
korea_monthly_scaled = pd.DataFrame(index=monthly_index)
china_monthly_scaled = pd.DataFrame(index=monthly_index)
japan_monthly_scaled = pd.DataFrame(index=monthly_index)

# 각 국가별 데이터 프레임 처리
for nation, df in [('korea', korea_df), ('china', china_df), ('japan', japan_df)]:
    df['created_utc'] = pd.to_datetime(df['created_utc'], errors='coerce')
    df = df[(df['created_utc'] >= start_date) & (df['created_utc'] <= end_date)]
    df['year_month'] = df['created_utc'].dt.to_period('M')
    df['weighted_score'] = df['score']
    df = df[df['label'].isin(valid_labels)]
    
    if df.empty:
        print(f"No data for {nation} with label {valid_labels}")
        continue
    
    monthly_weighted = df.groupby(['year_month', 'label'])['weighted_score'].sum().unstack(fill_value=0)
    monthly_scaled = pd.DataFrame(MinMaxScaler().fit_transform(monthly_weighted),
                                  index=monthly_weighted.index,
                                  columns=monthly_weighted.columns)
    
    # 월별 인덱스와 병합
    monthly_scaled = monthly_scaled.reindex(monthly_index, fill_value=0)
    
    if nation == 'korea':
        korea_monthly_scaled = monthly_scaled
    elif nation == 'china':
        china_monthly_scaled = monthly_scaled
    elif nation == 'japan':
        japan_monthly_scaled = monthly_scaled

# 그래프 설정
plt.figure(figsize=(14, 8))

# 국가별 anger 감정의 월별 가중치 점수 그래프 그리기
if not korea_monthly_scaled.empty:
    plt.plot(korea_monthly_scaled.index.astype(str), korea_monthly_scaled['joy'], marker='o', label='Korea')
if not china_monthly_scaled.empty:
    plt.plot(china_monthly_scaled.index.astype(str), china_monthly_scaled['joy'], marker='o', label='China')
if not japan_monthly_scaled.empty:
    plt.plot(japan_monthly_scaled.index.astype(str), japan_monthly_scaled['joy'], marker='o', label='Japan')

# 배경 색상 설정
ax = plt.gca()
ax.axvspan('2018-01', '2020-01', color='lightblue', alpha=0.3)
ax.axvspan('2020-01', '2022-01', color='lightpink', alpha=0.3)
ax.axvspan('2022-01', '2023-12', color='lightgreen', alpha=0.3)

# 그래프 제목 및 축 레이블 설정
plt.title('Monthly Weighted Scores of Joy (Scaled)')
plt.xlabel('Year-Month')
plt.ylabel('Scaled Weighted Score')
plt.legend(title='Country')
plt.grid(True)

# x축 레이블을 90도 회전
plt.xticks(rotation=90)

# 그래프 출력
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# 시작 날짜와 끝 날짜를 지정합니다.
start_date = '2018-01-01'
end_date = '2023-12-31'
valid_labels = ['anger']  # 표시할 감정을 'surprise'로 변경

# 월별 인덱스 생성
monthly_index = pd.period_range(start=start_date, end=end_date, freq='M')

# 각 국가별 데이터 프레임 초기화
korea_monthly_scaled = pd.DataFrame(index=monthly_index)
china_monthly_scaled = pd.DataFrame(index=monthly_index)
japan_monthly_scaled = pd.DataFrame(index=monthly_index)

# 각 국가별 데이터 프레임 처리
for nation, df in [('korea', korea_df), ('china', china_df), ('japan', japan_df)]:
    df['created_utc'] = pd.to_datetime(df['created_utc'], errors='coerce')
    df = df[(df['created_utc'] >= start_date) & (df['created_utc'] <= end_date)]
    df['year_month'] = df['created_utc'].dt.to_period('M')
    df['weighted_score'] = df['score']
    df = df[df['label'].isin(valid_labels)]
    
    if df.empty:
        print(f"No data for {nation} with label {valid_labels}")
        continue
    
    monthly_weighted = df.groupby(['year_month', 'label'])['weighted_score'].sum().unstack(fill_value=0)
    monthly_scaled = pd.DataFrame(MinMaxScaler().fit_transform(monthly_weighted),
                                  index=monthly_weighted.index,
                                  columns=monthly_weighted.columns)
    
    # 월별 인덱스와 병합
    monthly_scaled = monthly_scaled.reindex(monthly_index, fill_value=0)
    
    if nation == 'korea':
        korea_monthly_scaled = monthly_scaled
    elif nation == 'china':
        china_monthly_scaled = monthly_scaled
    elif nation == 'japan':
        japan_monthly_scaled = monthly_scaled

# 그래프 설정
plt.figure(figsize=(14, 8))

# 국가별 anger 감정의 월별 가중치 점수 그래프 그리기
if not korea_monthly_scaled.empty:
    plt.plot(korea_monthly_scaled.index.astype(str), korea_monthly_scaled['anger'], marker='o', label='Korea')
if not china_monthly_scaled.empty:
    plt.plot(china_monthly_scaled.index.astype(str), china_monthly_scaled['anger'], marker='o', label='China')
if not japan_monthly_scaled.empty:
    plt.plot(japan_monthly_scaled.index.astype(str), japan_monthly_scaled['anger'], marker='o', label='Japan')

# 배경 색상 설정
ax = plt.gca()
ax.axvspan('2018-01', '2020-01', color='lightblue', alpha=0.3)
ax.axvspan('2020-01', '2022-01', color='lightpink', alpha=0.3)
ax.axvspan('2022-01', '2023-12', color='lightgreen', alpha=0.3)

# 그래프 제목 및 축 레이블 설정
plt.title('Monthly Weighted Scores of Anger (Scaled)')
plt.xlabel('Year-Month')
plt.ylabel('Scaled Weighted Score')
plt.legend(title='Country')
plt.grid(True)

# x축 레이블을 90도 회전
plt.xticks(rotation=90)

# 그래프 출력
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# 시작 날짜와 끝 날짜를 지정합니다.
start_date = '2018-01-01'
end_date = '2023-12-31'
valid_labels = ['fear']  # 표시할 감정을 'surprise'로 변경

# 월별 인덱스 생성
monthly_index = pd.period_range(start=start_date, end=end_date, freq='M')

# 각 국가별 데이터 프레임 초기화
korea_monthly_scaled = pd.DataFrame(index=monthly_index)
china_monthly_scaled = pd.DataFrame(index=monthly_index)
japan_monthly_scaled = pd.DataFrame(index=monthly_index)

# 각 국가별 데이터 프레임 처리
for nation, df in [('korea', korea_df), ('china', china_df), ('japan', japan_df)]:
    df['created_utc'] = pd.to_datetime(df['created_utc'], errors='coerce')
    df = df[(df['created_utc'] >= start_date) & (df['created_utc'] <= end_date)]
    df['year_month'] = df['created_utc'].dt.to_period('M')
    df['weighted_score'] = df['score']
    df = df[df['label'].isin(valid_labels)]
    
    if df.empty:
        print(f"No data for {nation} with label {valid_labels}")
        continue
    
    monthly_weighted = df.groupby(['year_month', 'label'])['weighted_score'].sum().unstack(fill_value=0)
    monthly_scaled = pd.DataFrame(MinMaxScaler().fit_transform(monthly_weighted),
                                  index=monthly_weighted.index,
                                  columns=monthly_weighted.columns)
    
    # 월별 인덱스와 병합
    monthly_scaled = monthly_scaled.reindex(monthly_index, fill_value=0)
    
    if nation == 'korea':
        korea_monthly_scaled = monthly_scaled
    elif nation == 'china':
        china_monthly_scaled = monthly_scaled
    elif nation == 'japan':
        japan_monthly_scaled = monthly_scaled

# 그래프 설정
plt.figure(figsize=(14, 8))

# 국가별 anger 감정의 월별 가중치 점수 그래프 그리기
if not korea_monthly_scaled.empty:
    plt.plot(korea_monthly_scaled.index.astype(str), korea_monthly_scaled['fear'], marker='o', label='Korea')
if not china_monthly_scaled.empty:
    plt.plot(china_monthly_scaled.index.astype(str), china_monthly_scaled['fear'], marker='o', label='China')
if not japan_monthly_scaled.empty:
    plt.plot(japan_monthly_scaled.index.astype(str), japan_monthly_scaled['fear'], marker='o', label='Japan')

# 배경 색상 설정
ax = plt.gca()
ax.axvspan('2018-01', '2020-01', color='lightblue', alpha=0.3)
ax.axvspan('2020-01', '2022-01', color='lightpink', alpha=0.3)
ax.axvspan('2022-01', '2023-12', color='lightgreen', alpha=0.3)

# 그래프 제목 및 축 레이블 설정
plt.title('Monthly Weighted Scores of Fear (Scaled)')
plt.xlabel('Year-Month')
plt.ylabel('Scaled Weighted Score')
plt.legend(title='Country')
plt.grid(True)

# x축 레이블을 90도 회전
plt.xticks(rotation=90)

# 그래프 출력
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# 시작 날짜와 끝 날짜를 지정합니다.
start_date = '2018-01-01'
end_date = '2023-12-31'
valid_labels = ['disgust']  # 표시할 감정을 'surprise'로 변경

# 월별 인덱스 생성
monthly_index = pd.period_range(start=start_date, end=end_date, freq='M')

# 각 국가별 데이터 프레임 초기화
korea_monthly_scaled = pd.DataFrame(index=monthly_index)
china_monthly_scaled = pd.DataFrame(index=monthly_index)
japan_monthly_scaled = pd.DataFrame(index=monthly_index)

# 각 국가별 데이터 프레임 처리
for nation, df in [('korea', korea_df), ('china', china_df), ('japan', japan_df)]:
    df['created_utc'] = pd.to_datetime(df['created_utc'], errors='coerce')
    df = df[(df['created_utc'] >= start_date) & (df['created_utc'] <= end_date)]
    df['year_month'] = df['created_utc'].dt.to_period('M')
    df['weighted_score'] = df['score']
    df = df[df['label'].isin(valid_labels)]
    
    if df.empty:
        print(f"No data for {nation} with label {valid_labels}")
        continue
    
    monthly_weighted = df.groupby(['year_month', 'label'])['weighted_score'].sum().unstack(fill_value=0)
    monthly_scaled = pd.DataFrame(MinMaxScaler().fit_transform(monthly_weighted),
                                  index=monthly_weighted.index,
                                  columns=monthly_weighted.columns)
    
    # 월별 인덱스와 병합
    monthly_scaled = monthly_scaled.reindex(monthly_index, fill_value=0)
    
    if nation == 'korea':
        korea_monthly_scaled = monthly_scaled
    elif nation == 'china':
        china_monthly_scaled = monthly_scaled
    elif nation == 'japan':
        japan_monthly_scaled = monthly_scaled

# 그래프 설정
plt.figure(figsize=(14, 8))

# 국가별 anger 감정의 월별 가중치 점수 그래프 그리기
if not korea_monthly_scaled.empty:
    plt.plot(korea_monthly_scaled.index.astype(str), korea_monthly_scaled['disgust'], marker='o', label='Korea')
if not china_monthly_scaled.empty:
    plt.plot(china_monthly_scaled.index.astype(str), china_monthly_scaled['disgust'], marker='o', label='China')
if not japan_monthly_scaled.empty:
    plt.plot(japan_monthly_scaled.index.astype(str), japan_monthly_scaled['disgust'], marker='o', label='Japan')

# 배경 색상 설정
ax = plt.gca()
ax.axvspan('2018-01', '2020-01', color='lightblue', alpha=0.3)
ax.axvspan('2020-01', '2022-01', color='lightpink', alpha=0.3)
ax.axvspan('2022-01', '2023-12', color='lightgreen', alpha=0.3)

# 그래프 제목 및 축 레이블 설정
plt.title('Monthly Weighted Scores of Disgust (Scaled)')
plt.xlabel('Year-Month')
plt.ylabel('Scaled Weighted Score')
plt.legend(title='Country')
plt.grid(True)

# x축 레이블을 90도 회전
plt.xticks(rotation=90)

# 그래프 출력
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# 시작 날짜와 끝 날짜를 지정합니다.
start_date = '2018-01-01'
end_date = '2023-12-31'
valid_labels = ['neutral']  # 표시할 감정을 'surprise'로 변경

# 월별 인덱스 생성
monthly_index = pd.period_range(start=start_date, end=end_date, freq='M')

# 각 국가별 데이터 프레임 초기화
korea_monthly_scaled = pd.DataFrame(index=monthly_index)
china_monthly_scaled = pd.DataFrame(index=monthly_index)
japan_monthly_scaled = pd.DataFrame(index=monthly_index)

# 각 국가별 데이터 프레임 처리
for nation, df in [('korea', korea_df), ('china', china_df), ('japan', japan_df)]:
    df['created_utc'] = pd.to_datetime(df['created_utc'], errors='coerce')
    df = df[(df['created_utc'] >= start_date) & (df['created_utc'] <= end_date)]
    df['year_month'] = df['created_utc'].dt.to_period('M')
    df['weighted_score'] = df['score']
    df = df[df['label'].isin(valid_labels)]
    
    if df.empty:
        print(f"No data for {nation} with label {valid_labels}")
        continue
    
    monthly_weighted = df.groupby(['year_month', 'label'])['weighted_score'].sum().unstack(fill_value=0)
    monthly_scaled = pd.DataFrame(MinMaxScaler().fit_transform(monthly_weighted),
                                  index=monthly_weighted.index,
                                  columns=monthly_weighted.columns)
    
    # 월별 인덱스와 병합
    monthly_scaled = monthly_scaled.reindex(monthly_index, fill_value=0)
    
    if nation == 'korea':
        korea_monthly_scaled = monthly_scaled
    elif nation == 'china':
        china_monthly_scaled = monthly_scaled
    elif nation == 'japan':
        japan_monthly_scaled = monthly_scaled

# 그래프 설정
plt.figure(figsize=(14, 8))

# 국가별 anger 감정의 월별 가중치 점수 그래프 그리기
if not korea_monthly_scaled.empty:
    plt.plot(korea_monthly_scaled.index.astype(str), korea_monthly_scaled['neutral'], marker='o', label='Korea')
if not china_monthly_scaled.empty:
    plt.plot(china_monthly_scaled.index.astype(str), china_monthly_scaled['neutral'], marker='o', label='China')
if not japan_monthly_scaled.empty:
    plt.plot(japan_monthly_scaled.index.astype(str), japan_monthly_scaled['neutral'], marker='o', label='Japan')

# 배경 색상 설정
ax = plt.gca()
ax.axvspan('2018-01', '2020-01', color='lightblue', alpha=0.3)
ax.axvspan('2020-01', '2022-01', color='lightpink', alpha=0.3)
ax.axvspan('2022-01', '2023-12', color='lightgreen', alpha=0.3)

# 그래프 제목 및 축 레이블 설정
plt.title('Monthly Weighted Scores of Neutral (Scaled)')
plt.xlabel('Year-Month')
plt.ylabel('Scaled Weighted Score')
plt.legend(title='Country')
plt.grid(True)

# x축 레이블을 90도 회전
plt.xticks(rotation=90)

# 그래프 출력
plt.tight_layout()
plt.show()